In [1]:
import os,sys
import numpy as np
import pandas as pd
# from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
# from tqdm import tqdm
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostRegressor
import gc
from  numba import cuda


print_to_file = False 
test_run = False 

# train = pd.read_pickle(ENV.lightgbm_train_764.value)
train = pd.read_pickle(ENV.lgb_train_0827_na.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(ENV.lightgbm_test_764.value)
test = pd.read_pickle(ENV.lgb_test_0827_na.value)
print('test shape is: {}'.format(test.shape))
fe_id = 'comb_764'

train shape is: (307511, 409)
test shape is: (48744, 408)


In [2]:
train['SK_ID_CURR'] = train['SK_ID_CURR'].astype(int)
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype(int)
targets = train.TARGET.values

train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

X = pd.concat([train.drop('TARGET',axis=1),test])

In [3]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [4]:
def fill_na_list(X,th=200):
    re = scan_nan_portion(X)
    re = re[re<0.15]
    re = re.sort_values()
    col_list = []
    na_list = []
    for col in re.index:
        if re[col] > 0 and X[col].nunique() >= th and str(X[col].dtypes)!= 'category' and str(X[col].dtypes)!= 'object':
            col_list.append(col)
            na_list.append(re[col])
    return col_list,na_list
            
def check_category_col(X,th=200):
    cate = []
    cols = list(X.columns)
    for col in cols:
        if X[col].nunique() < th or str(X[col].dtypes)== 'category' or str(X[col].dtypes)== 'object':
            cate.append(cols)
    return cate


def cate_index(df,category):
    index_list = []
    for col in df.columns:
        if col in category:
            index_list.append(category.index(col))
    return index_list
    
def trainCAT(na_train_x,na_val_x,na_train_y,na_val_y):
    cat_col = cate_index(na_train_x,categorys)
    model = CatBoostRegressor(iterations=6000, 
                              depth=None, 
                              thread_count=10,
                              learning_rate=0.01, 
                              loss_function='RMSE',
                              verbose=500,
                              task_type='GPU')
    model.fit(na_train_x,na_train_y,
              cat_features=cat_col,
              eval_set=[(na_val_x, na_val_y)],
              early_stopping_rounds=3,
              verbose_eval=500,
              metric_period=500)

#     reg.fit(na_train_x, na_train_y, eval_set=[(na_train_x,na_train_y),(na_val_x, na_val_y)],  verbose=200,early_stopping_rounds=100,eval_metric='l1' )
    return model

def saving(X):
    print('saving...')
    train_save = X.iloc[:307511].copy()
    train_save['TARGET'] = targets
    print(train_save.shape)

    test_save = X.iloc[307511:].copy()
    train_save.to_pickle(ENV.lgb_train_0827_na.value)
    test_save.to_pickle(ENV.lgb_test_0827_na.value)
    

In [5]:
col_list,na_list = fill_na_list(X)
categorys = check_category_col(X)

In [6]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col))
val = 0.1
save_count = 1
count = 0
for col,nav in zip(col_list,na_list):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
#     feature_col = list(set(feature_col_base + Processed_list))
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainCAT(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    del reg
    gc.collect()
    

start processing bureau_debt_credit_ratio 
the na is:0.14499445621815835
(304535, 406)
(51720, 406)
0:	learn: 0.5238301	test: 0.3995736	best: 0.3995736 (0)	total: 11.9ms	remaining: 1m 11s
500:	learn: 0.1828863	test: 0.0961590	best: 0.0961590 (500)	total: 4.81s	remaining: 52.8s
1000:	learn: 0.1037469	test: 0.1301839	best: 0.0961590 (500)	total: 9.07s	remaining: 45.3s
1500:	learn: 0.0684174	test: 0.1457059	best: 0.0961590 (500)	total: 13.4s	remaining: 40.3s
2000:	learn: 0.0525491	test: 0.1456793	best: 0.0961590 (500)	total: 17.9s	remaining: 35.8s
bestTest = 0.09615903815
bestIteration = 500
processed: 1/6
saving...
(307511, 409)
start processing Wei_Remaining_AMT_Payment_LAST 
the na is:0.14725968758333216
(303793, 406)
(52462, 406)
0:	learn: 312529.8232115	test: 296503.4561384	best: 296503.4561384 (0)	total: 9.63ms	remaining: 57.8s
500:	learn: 123104.0622801	test: 124070.4239632	best: 124070.4239632 (500)	total: 4.21s	remaining: 46.2s
1000:	learn: 106562.3232799	test: 110387.4852845	bes

In [ ]:
bureau_AMT_CREDIT_SUM_divide_DAYS_CREDIT_ENDDATE_min 